In [1]:
from google.colab import drive
import os
import cv2
import numpy as np
from sklearn.utils import shuffle
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Mount Google Drive
drive.mount('/content/drive')

# Image size
size = 100

# Load images
def load_images_from_folder(folder_path, size):
    image_paths = [os.path.join(folder_path, img)
                   for img in os.listdir(folder_path)
                   if img.lower().endswith(('.jpg', '.jpeg', '.png'))]
    images = [cv2.resize(cv2.imread(path), (size, size)) for path in image_paths if cv2.imread(path) is not None]
    return np.array(images)

# Load datasets
pothole_train = load_images_from_folder("/content/drive/MyDrive/Dataset/train/Pothole", size)
plain_train = load_images_from_folder("/content/drive/MyDrive/Dataset/train/Plain", size)
pothole_test = load_images_from_folder("/content/drive/MyDrive/Dataset/test/Pothole", size)
plain_test = load_images_from_folder("/content/drive/MyDrive/Dataset/test/Plain", size)

# Combine data
X_train = np.concatenate((pothole_train, plain_train))
X_test = np.concatenate((pothole_test, plain_test))
y_train = np.concatenate((np.ones(len(pothole_train), dtype=int), np.zeros(len(plain_train), dtype=int)))
y_test = np.concatenate((np.ones(len(pothole_test), dtype=int), np.zeros(len(plain_test), dtype=int)))

# Shuffle
X_train, y_train = shuffle(X_train, y_train, random_state=42)
X_test, y_test = shuffle(X_test, y_test, random_state=42)

# Normalize images
X_train = X_train / 255.0
X_test = X_test / 255.0

# One-hot encode labels
y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)

# Image augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.7, 1.3],
    fill_mode='nearest',
    validation_split=0.1  # 10% validation split
)

# Data generators
train_generator = datagen.flow(X_train, y_train, batch_size=32, subset='training')
val_generator = datagen.flow(X_train, y_train, batch_size=32, subset='validation')

# Define model using VGG16
def kerasModelVGG16(size=100):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(size, size, 3))
    model = Sequential()
    model.add(base_model)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(2, activation='softmax'))
    for layer in base_model.layers:
        layer.trainable = False
    return model

# Compile model
model = kerasModelVGG16(size)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
history = model.fit(train_generator, validation_data=val_generator, epochs=25)

# Evaluate on test data
metrics = model.evaluate(X_test, y_test)
for name, value in zip(model.metrics_names, metrics):
    print(f"{name}: {value:.4f}")

# Save model
print("Saving model weights and configuration file")
model.save('sample_vgg16_augmented.h5')
model_json = model.to_json()
with open("sample_vgg16_augmented.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("sample_vgg16_augmented.weights.h5")
print("Saved model to disk")

# Predict
y_pred_probs = model.predict(X_test)
y_pred_classes = np.argmax(y_pred_probs, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Evaluation metrics
print("\nClassification Report:")
print(classification_report(y_true_classes, y_pred_classes, target_names=["Plain", "Pothole"]))

f1 = f1_score(y_true_classes, y_pred_classes)
print("F1 Score (binary):", f1)

cm = confusion_matrix(y_true_classes, y_pred_classes)
print("\nConfusion Matrix:")
print(cm)


Mounted at /content/drive
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 91s 4s/step - accuracy: 0.4799 - loss: 0.7146 - val_accuracy: 0.4306 - val_loss: 0.6955
Epoch 2/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 90s 4s/step - accuracy: 0.4775 - loss: 0.7089 - val_accuracy: 0.4444 - val_loss: 0.6978
Epoch 3/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 129s 4s/step - accuracy: 0.5179 - loss: 0.7071 - val_accuracy: 0.5694 - val_loss: 0.6853
Epoch 4/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 77s 4s/step - accuracy: 0.4790 - loss: 0.7041 - val_accuracy: 0.4444 - val_loss: 0.6870
Epoch 5/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 77s 4s/step - accuracy: 0.5149 - loss: 0.6968 - val_accuracy: 0.4444 - val_loss: 0.7006
Epoch 6/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 80s 4s/step - accuracy: 0.5135 - loss: 0.7080 - val_accuracy: 0.4444 - val_loss: 0.6927
Epoch 7/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 76s 4s/step - accuracy: 0.5397 - loss: 0.6903 - val_accuracy: 0.5556 - val_loss: 0.6888
Epoch 8/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 76s 4s/step - accuracy: 0.4825 - loss: 0.7068 - val_accuracy: 0.4444 - val_loss

loss: 0.8613
compile_metrics: 0.5000
Saving model weights and configuration file
Saved model to disk
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step

Classification Report:
              precision    recall  f1-score   support

       Plain       0.50      1.00      0.67         8
     Pothole       0.00      0.00      0.00         8

    accuracy                           0.50        16
   macro avg       0.25      0.50      0.33        16
weighted avg       0.25      0.50      0.33        16

F1 Score (binary): 0.0

Confusion Matrix:
[[8 0]
 [8 0]]


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
